In [1]:
# Import necessary packages

import numpy as np
import opendssdirect as dss
# from utils.device.Inverter import Inverter
# from utils.controller.AdaptiveInvController import AdaptiveInvController
# from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

from collections import deque
import signal_processing as signal_processing

# import numpy as np
# from matplotlib.mlab import PCA
# from sklearn.decomposition import PCA

import scipy.linalg as spla

In [2]:
# Set global variables and parameters, and test OpenDSS with network


#######################################################
###Global variable initialization and error checking###
#######################################################

Sbase=1
LoadScalingFactor = 3
GenerationScalingFactor = 5
SlackBusVoltage = 1.00
NoiseMultiplyer= 1
#Set simulation analysis period - the simulation is from StartTime to EndTime
StartTime = 40600
EndTime = StartTime + 640
EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
#Set hack parameters
TimeStepOfHack = 160
PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

#Set initial VBP parameters for uncompromised inverters
VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

#Set delays for each node
Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
#Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
lpf_meas_vector = (1+0.2*np.random.randn(31))
lpf_output_vector= (0.1+0.015*np.random.randn(31))

#Set observer voltage threshold
ThreshHold_vqvp = 0.06
adaptive_gain=800


power_factor=0.9
pf_converted=tan(acos(power_factor))
# Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters


#Error checking of the global variable -- TODO: add error handling here!
if EndTime < StartTime or EndTime < 0 or StartTime < 0:
    print('Setup Simulation Times Inappropriately.')
if NoiseMultiplyer < 0:
    print('Setup Noise Multiplyer Correctly.')

# Select OpenDSS file
dss.run_command('Redirect 05node_singlephase_balanced_oscillation.dss')

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print('')

# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')

    
# dir(dss.Bus)

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.

7
['sourcebus', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06']
16
['load_a02_a', 'load_a03_a', 'load_a04_a', 'load_a06_a', 'inv_a01_a', 'inv_a02_a', 'inv_a03_a', 'inv_a04_a', 'inv_a05_a', 'inv_a06_a', 'esc_a01_a', 'esc_a02_a', 'esc_a03_a', 'esc_a04_a', 'esc_a05_a', 'esc_a06_a']

sourcebus
BASE - LN
132.79056191361394
VMAGANGLE - LN POLAR
[132790.33086352702, -0.00014308484621030246, 132790.54105729682, -120.00000345245778, 132790.57930807228, 119.99999394344195]
puVMAGANGLE - LN POLAR
[0.9999982600413495, -0.00014308484621030246, 0.999999842938257, -120.00000345245778, 1.0000001309916766, 119.99999394344195]
VOTLAGES - LN CARTESIAN
[132790.33086311293, -0.33161751585013377, -66395.27747110755, -114999.97792966862, -66395.27751067649, 115000.02207501393]
puVOTLAGES - LN CARTESIAN
[0.9999982600382311, -2.497297331009605e-06, -0.49999997375039784, -0.8660252375803721, -0.4999999740483779, 0.8660255700237678]
VLL

In [3]:
# Inverter object class


class inverter():
    
    def __init__(self):
        
        #
        self.max_reactive_capacity = 1
        self.reactive_capacity = 1
    
        # Current reactive power
        self.reactive_power = 0
    
        # Volt-VAr curve breakpoints
        self.VBP = np.array([0.97, 0.99, 1.01, 1.03])
        
        # Lowpass filter cutoff frequency
        self.wlp = 2*np.pi*0.5
        
        # Lowpass filter timestep
        self.Ts = 1.0
        
        # Measured voltage
        self.Vmeas = np.zeros(86401)
        
        # Lowpass filter voltage
        self.Vlp = np.zeros(86401)
        
        # Array of reactive power
        self.reactive_power = np.zeros(86401)
        
        # attack status
        self.hackFlag = False
        
    def set_timesteps(self, startTimeStep, endTimeStep):
        self.startTimeStep = startTimeStep
        self.endTimeStep = endTimeStep
        
        self.Vmeas = np.zeros(endTimeStep - startTimeStep)
        self.Vlp = np.zeros(endTimeStep - startTimeStep)
        self.reactive_power = np.zeros(endTimeStep - startTimeStep)
        
    # set the index of the node where the inverter is located
    def set_nodenum(self, nodenum):
        self.nodenum = nodenum
        
    # set the name of the node where the inverter is located
    def set_nodename(self, nodename):
        self.nodename = nodename
        
    # set the name of the node where the inverter is located
    def set_loadname(self, loadname):
        self.loadname = loadname
        
    def set_VAr_capacity(self, VAr_capacity):
        self.VAr_capacity = VAr_capacity
    
    # set Volt-VAr curve breakpoints
    def set_VBP(self, VBP):
        self.VBP = VBP
    
    # set measured voltage
    def measure_voltage(self, timeStep, Vmeas):
        self.Vmeas[kt] = Vmeas
    
    # lowpass filter voltage measurements
    def lowpass(self, kt):
        if kt == 0:
            self.Vlp[kt] = self.Vmeas[kt]
        else:
            self.Vlp[kt] = (1 - self.wlp*self.Ts)*self.Vlp[kt-1] + self.wlp*self.Ts*self.Vmeas[kt-1]
            self.Vlp[kt] = 1/(2 + self.wlp*self.Ts)*((2 - self.wlp*self.Ts)*self.Vlp[kt-1] + self.wlp*self.Ts*(self.Vmeas[kt] + self.Vmeas[kt-1]))
    
    # compute reactive power from lowpass filtered voltage and Volt-Var curve
    def compute_reactive_power_output(self, kt, Vcomp):
        if Vcomp <= self.VBP[0]:
            self.reactive_power[kt] = -100
        elif self.VBP[0] <= Vcomp <= self.VBP[1]:
            self.reactive_power[kt] =100/(self.VBP[1] - self.VBP[0])*(Vcomp - self.VBP[1])
        elif self.VBP[1] <= Vcomp <= self.VBP[2]:
            self.reactive_power[kt] = 0
        elif self.VBP[2] <= Vcomp <= self.VBP[3]:
            self.reactive_power[kt] = 100/(self.VBP[3] - self.VBP[2])*(Vcomp - self.VBP[2])
        elif self.VBP[3] <= Vcomp:
            self.reactive_power[kt] = 100
        # print(self.reactive_power)
        
    

x1 = inverter()
x1.set_VBP(np.array([0.97, 0.99, 1.01, 1.03]))
print(x1.VBP)

x1.compute_reactive_power_output(0,1.02)
print(x1.reactive_power)

print(x1.Vmeas)
print(x1.Vlp)

[0.97 0.99 1.01 1.03]
[50.  0.  0. ...  0.  0.  0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [4]:

class esc():
    
    def __init__(self):
    
        # init for signal processing on Voltage
        self.Ts = 1
        self.fosc = 0.15
        self.hp1, temp = signal_processing.butterworth_highpass(2,2*np.pi*0.5*self.fosc)
        self.lp1, temp = signal_processing.butterworth_lowpass(4,2*np.pi*2*self.fosc)
        self.bp1num = np.convolve(self.hp1[0, :], self.lp1[0, :])
        self.bp1den = np.convolve(self.hp1[1, :], self.lp1[1, :])
        self.bp1s = np.array([self.bp1num, self.bp1den])
#         self.bp1s = self.hp1
        self.BP1z = signal_processing.c2dbilinear(self.bp1s, self.Ts)
        self.lpf2, temp = signal_processing.butterworth_lowpass(2,2*np.pi*self.fosc/2)
        self.LPF2z = signal_processing.c2dbilinear(self.lpf2, self.Ts)
        self.nbp1 = self.BP1z.shape[1] - 1
        self.nlpf2 = self.LPF2z.shape[1] - 1

#         self.x = deque([0]*(len(self.BP1z[0, :]) + step_buffer*2), maxlen=(len(self.BP1z[0, :]) + step_buffer*2))
#         self.y1 = deque([0]*len(self.BP1z[1, 0:-1]), maxlen=len(self.BP1z[1, 0:-1]))
#         self.y2 = deque([0]*len(self.LPF2z[0, :]), maxlen=len(self.LPF2z[0, :]))
#         self.y3 = deque([0]*len(self.LPF2z[1, 0:-1]), maxlen=len(self.LPF2z[1, 0:-1]))

        self.fes = 0.1
        self.wes = 2*np.pi*self.fes
        self.aes = 20
        self.wh = self.wes/10
        self.wl = self.wes/10
        self.kes = 5e5
        
        
    def set_timesteps(self, startTimeStep, endTimeStep):
        self.startTimeStep = startTimeStep
        self.endTimeStep = endTimeStep
        
        self.x = np.zeros(endTimeStep - startTimeStep)
        self.y1 = np.zeros(endTimeStep - startTimeStep)
        self.y2 = np.zeros(endTimeStep - startTimeStep)
        self.y3 = np.zeros(endTimeStep - startTimeStep)
        self.y4 = np.zeros(endTimeStep - startTimeStep)
        
        self.psi = np.zeros(endTimeStep - startTimeStep)
        self.rho = np.zeros(endTimeStep - startTimeStep)
        self.eps = np.zeros(endTimeStep - startTimeStep)
        self.sigma = np.zeros(endTimeStep - startTimeStep)
        self.xi = np.zeros(endTimeStep - startTimeStep)
        self.uhat = np.zeros(endTimeStep - startTimeStep)
        self.u = np.zeros(endTimeStep - startTimeStep)
        
    def set_esc_params(self, fes, aes, kes):
        self.fes = fes
        self.wes = 2*np.pi*self.fes
        self.aes = aes
        self.wh = self.wes/10
        self.wl = self.wes/10
        self.kes = kes
        
    # set the index of the node where the inverter is located
    def set_nodenum(self, nodenum):
        self.nodenum = nodenum
        
    # set the name of the node where the inverter is located
    def set_nodename(self, nodename):
        self.nodename = nodename
        
    # set the name of the node where the inverter is located
    def set_loadname(self, loadname):
        self.loadname = loadname
        
    def observer(self, kt, vk):
        
#         vk = np.abs(k.node.nodes[node_id]['voltage'][k.time - 1])
#         vkm1 = np.abs(k.node.nodes[node_id]['voltage'][k.time - 2])
#         self.v_meas_k = vk
#         self.v_meas_km1 = vkm1
        self.x[kt] = vk
    
        if kt >= self.BP1z.shape[1]:

            np.sum(-self.BP1z[1,0:-1]*self.y1[kt-self.BP1z.shape[1]+1:kt])
            np.sum(self.BP1z[0,:]*self.x[kt-self.BP1z.shape[1]+1:kt+1])
            
            self.y1[kt] = (1/self.BP1z[1,-1]*(np.sum(-self.BP1z[1,0:-1]*self.y1[kt-self.BP1z.shape[1]+1:kt]) + np.sum(self.BP1z[0,:]*self.x[kt-self.BP1z.shape[1]+1:kt+1])))
            self.y2[kt] = (self.y1[kt]**2)
            self.y3[kt] = (1/self.LPF2z[1,-1]*(np.sum(-self.LPF2z[1,0:-1]*self.y3[kt-self.LPF2z.shape[1]+1:kt]) + np.sum(self.LPF2z[0,:]*self.y2[kt-self.LPF2z.shape[1]+1:kt+1])))
            self.y4[kt] = np.sqrt(np.abs(self.y3[kt]))
            
    def esc_function(self, kt):
        
        if kt >= 1:
            
            self.rho[kt] = (1 - self.Ts*self.wh)*self.rho[kt-1] + self.y4[kt] - self.y4[kt-1]
            self.rho[kt] = 1/(2 + self.Ts*self.wh)*((2 - self.Ts*self.wh)*self.rho[kt-1] + 2*(self.y4[kt] - self.y4[kt-1]))
            
            self.eps[kt] = self.y4[kt] - self.rho[kt]
            
            self.sigma[kt] = 2/self.aes*np.sin(self.wes*time[kt-1])*self.rho[kt]
            
            self.xi[kt] = (1 - self.Ts*self.wl)*self.xi[kt-1] + self.Ts*self.wl*self.sigma[kt-1]
            self.xi[kt] = 1/(2 + self.Ts*self.wl)*((2 - self.Ts*self.wl)*self.xi[kt-1] + self.Ts*self.wl*(self.sigma[kt] + self.sigma[kt-1]))
            
            self.uhat[kt] = self.uhat[kt-1] - 1*self.Ts*self.kes*self.xi[kt-1]
            self.uhat[kt] = self.uhat[kt-1] - 1/2*self.Ts*self.kes*(self.xi[kt] + self.xi[kt-1])
            
            self.u[kt] = self.uhat[kt] + self.aes*np.sin(self.wes*time[kt])
        

In [5]:
# Run simulation

dss.run_command('Redirect 05node_singlephase_balanced_oscillation.dss')
dss.Solution.Solve()


if dss.Solution.Converged:
    print('Initial Model Converged. Proceeding to Next Step.')
    
else:
    print('Initial Solution Not Converged. Check Model for Convergence')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

    

# Set slack bus (sourcebus) voltage reference in p.u. 
dss.Vsources.PU(SlackBusVoltage)

############################################
############ INIT NODES ####################
############################################
""" 
 nodes' variable is a dictionary contains all the nodes in the grid,
 with KEY is the node's number, VALUE is a dataframe with 
 ROW is ['Voltage', 'Generation', 'P', 'Q'] - Voltage is Voltage for each timestep; 
                                              P,Q is P,Q injection at that node for that timestep.
 and 
 COLUMN is each timestep for the whole simulation. 
 nodes = {
    1: dataFrame1,
    2: dataFrame2
 }
"""

nodes = {}
features = ['Voltage', 'Generation', 'P', 'Q']

# for k1 in range(len(AllLoadNames)):
#     print(k1)
#     df = pd.DataFrame(columns=list(range(len(time))),index=features)
#     nodes[k1] = df
#     nodes[k1].loc['Generation'] = Generation[:,k1]
#     nodes[k1].loc['Load'] = Load[:,k1]
#     nodes[k1].loc['P'] = 0
#     nodes[k1].loc['Q'] = 0
#     nodes[k1].loc['Pinj'] = 0
#     nodes[k1].loc['Qinj'] = 0
    

    
################################################
############ INIT INVERTERS ####################
################################################
""" 
 inverters' variable is a dictionary contains all the inverters in the grid,
 with KEY is the node's number where we have inverters, VALUE is a list of inverters at that node.
 
 Each inverter has a dictionary:
     'device': Inverter_Object
     'controller: Controller_Object
     'info': contain scenario information in a dataframe (solar generation, sbar) for each timestep at that node
 }
"""

# time array
Ts = 1.0
time = np.arange(0,120+Ts,Ts)

# All buses and bus voltages
AllBuses = dss.Circuit.AllBusNames()
AllBusVoltage = np.zeros((len(time),len(AllBuses)))

# All loads and load voltages
LoadBuses = dss.Loads.AllNames()
LoadBusVoltage = np.zeros((len(time),len(LoadBuses)))



##################################################
# Set up inverters
##################################################

# Create inverters and assign to existing load buses
# inverters = []
# for k1 in range(len(LoadBuses)):
#     if LoadBuses[k1] == 's701a' or LoadBuses[k1] == 's702a':
# #         for k2 in range(generation_indexes.shape[1]):
#         for k2 in range(0,1):

#     #         inverters[k1] = []
#             tempinverter = inverter()
#             tempinverter.set_timesteps(0,len(time))
#             tempinverter.set_nodenum(k1)
#             tempinverter.set_nodename(LoadBuses[k1])
#             tempinverter.set_VBP(np.array([0.95, 0.97, 1.03, 1.05]))
#             tempinverter.set_VBP(np.array([0.95, 0.99, 1.01, 1.05]))

#             inverters.append(tempinverter)

####################
####################
invbuslist = ['a03','a05']
            
inverters = []
for k1 in range(len(invbuslist)):
    tempinverter = inverter()
    tempinverter.set_timesteps(0,len(time))
#     tempinverter.set_nodenum(k1)
    tempinverter.set_nodename(invbuslist[k1])
    tempinverter.set_loadname('inv_' + invbuslist[k1] + '_a')
    tempinverter.set_VBP(np.array([0.95, 0.98, 1.02, 1.05]))

    inverters.append(tempinverter)

####################
####################

escbuslist = ['a03','a06']
escparams = np.array([[0.1, 0.09],[20, 20],[5e5, 5e5]])

escbuslist = ['a03']
escparams = np.array([[0.1],[20],[5e5]])

escs = []

for k1 in range(len(escbuslist)):
    tempesc = esc()
    tempesc.set_timesteps(0,len(time))
    tempesc.set_esc_params(escparams[0,k1],escparams[1,k1],escparams[2,k1])
    tempesc.set_nodename(escbuslist[k1])
    tempesc.set_loadname('esc_' + escbuslist[k1] + '_a')
    escs.append(tempesc)
    
##################################################
# ?
##################################################

# All buses and bus voltages
AllBuses = dss.Circuit.AllBusNames()
AllBusVoltage = np.zeros((len(time),len(AllBuses)))

# All loads and load voltages
LoadBuses = dss.Loads.AllNames()
LoadBusVoltage = np.zeros((len(time),len(LoadBuses)))

#
LoadBusPinj = np.zeros((len(time),len(LoadBuses))) 
LoadBusQinj = np.zeros((len(time),len(LoadBuses)))

kWloadTS = np.zeros((len(time),len(LoadBuses)))
kVArloadTS = np.zeros((len(time),len(LoadBuses)))

kWtotalTS = np.zeros((len(time),len(LoadBuses)))
kVArtotalTS = np.zeros((len(time),len(LoadBuses)))

# Real power load, injection, total for load buses
PloadTS = np.zeros((len(time),len(LoadBuses)))
PinvTS = np.zeros((len(time),len(LoadBuses)))
PescTS = np.zeros((len(time),len(LoadBuses)))
PtotalTS = np.zeros((len(time),len(LoadBuses)))

# Reactive power load, injection, total for load buses
QloadTS = np.zeros((len(time),len(LoadBuses)))
QinvTS = np.zeros((len(time),len(LoadBuses)))
QescTS = np.zeros((len(time),len(LoadBuses)))
QtotalTS = np.zeros((len(time),len(LoadBuses)))

# Nominal real and reactive power loads in kW and kVAr, from OpenDSS file
PloadNominal = np.zeros(len(LoadBuses))
QloadNominal = np.zeros(len(LoadBuses))
# Array of frequencies for load modulation
lf = np.zeros(len(LoadBuses))
for k1 in range(len(LoadBuses)):
    dss.Loads.Name(LoadBuses[k1])
    PloadNominal[k1] = dss.Loads.kW()
    QloadNominal[k1] = dss.Loads.kvar()
    lf[k1] = 1 + 0.2*(np.random.rand()-0.5)
    
print(LoadBuses)
print(PloadNominal)
print(QloadNominal)
print(lf)
print('')

# iterate through timesteps
for kt in range(0,len(time)):
       
    # print when each minute has elapsed
    if time[kt] % 60 == 0:
        hh = int((time[kt])/3600)
        mm = int(((time[kt]) - 3600*int((time[kt])/3600))/60)
        if mm <= 9:
            print(str(hh) + '.0' + str(mm))
        else:
            print(str(hh) + '.' + str(mm))
    
    # At first time step, initialize loads, with no control
    if kt == 0:
        for k1 in range(len(LoadBuses)):
            
            if 'load' in LoadBuses[k1]:
                dss.Loads.Name(LoadBuses[k1])
                
                PloadTS[kt,k1] = PloadNominal[k1]
                dss.Loads.kW(PloadTS[kt,k1])
                
                QloadTS[kt,k1] = QloadNominal[k1]
                dss.Loads.kvar(QloadTS[kt,k1])
                
#             if 'inv' in LoadBuses[k1]:
#                 dss.Loads.Name(LoadBuses[k1])                
                
#                 PinvTS[kt,k1] = 0
#                 dss.Loads.kW(PinvTS[kt,k1])
                
#                 QinvTS[kt,k1] = 0
#                 dss.Loads.kvar(QinvTS[kt,k1])
                
#             if 'esc' in LoadBuses[k1]:
#                 dss.Loads.Name(LoadBuses[k1])
                
#                 PescTS[kt,k1] = 0
#                 dss.Loads.kW(PescTS[kt,k1])
                
#                 QescTS[kt,k1] = 0
#                 dss.Loads.kvar(QescTS[kt,k1])

    # Compute total loading
    else:
        for k1 in range(len(LoadBuses)):
            
            if 'load' in LoadBuses[k1]:
                dss.Loads.Name(LoadBuses[k1])
                
                PloadTS[kt,k1] = PloadNominal[k1] + 15*np.sin(2*np.pi*lf[k1]*1/120*time[kt]) + 0*15*np.random.rand()
                dss.Loads.kW(PloadTS[kt,k1])
                
                QloadTS[kt,k1] = QloadNominal[k1] + 15*np.sin(2*np.pi*lf[k1]*1/120*time[kt]) + 0*10*np.random.rand()
                dss.Loads.kvar(QloadTS[kt,k1])
                
            for k1 in range(len(inverters)):            
                dss.Loads.Name(inverters[k1].loadname)                

                PinvTS[kt,k1] = 0
                dss.Loads.kW(PinvTS[kt,k1])

                QinvTS[kt,k1] = 1/100*200*inverters[k1].reactive_power[kt-1]
                dss.Loads.kvar(QinvTS[kt,k1])
                
            for k1 in range(len(escs)):
                dss.Loads.Name(escs[k1].loadname)
                
                PescTS[kt,k1] = 0
                dss.Loads.kW(PescTS[kt,k1])
                
                QescTS[kt] = escs[k1].u[kt-1]
#                 dss.Loads.kvar(QescTS[kt,k1])
                
#             if 'inv' in LoadBuses[k1]:
#                 dss.Loads.Name(LoadBuses[k1])                
                
#                 PinvTS[kt,k1] = 0
#                 dss.Loads.kW(PinvTS[kt,k1])
                
#                 QinvTS[kt,k1] = 0
#                 dss.Loads.kvar(QinvTS[kt,k1])
                
#             if 'esc' in LoadBuses[k1]:
#                 dss.Loads.Name(LoadBuses[k1])
                
#                 PescTS[kt,k1] = 0
#                 dss.Loads.kW(PescTS[kt,k1])
                
#                 QescTS[kt,k1] = 0
#                 dss.Loads.kvar(QescTS[kt,k1])

    # solve power flow with OpenDSS file and newly assigned loads
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Solution Not Converged at Step:', kt)
        
    dss.Monitors.SampleAll()
    #get the voltage info
    #(voltage[0]+voltage[2]+voltage[4])/(DSSCircuit.ActiveElement.NumPhases*(Loads.kV*1000/(3**0.5)))
    nodeInfo = []
    
    for k1 in range(len(LoadBuses)):
        dss.Loads.Name(LoadBuses[k1])
        voltage = dss.CktElement.VoltagesMagAng()
#         print(dss.CktElement.NodeOrder())
#         print(voltage)
#         voltagePU = (voltage[0]+voltage[2]+voltage[4])/(dss.CktElement.NumPhases()*(dss.Loads.kV()*1000/(3**0.5)))

#         LoadBusVoltage[kt,k1] = voltagePU
        
#     for k1 in range(len(AllBuses)):
# #         dss.Loads.Name(LoadBuses[k1])
#         dss.Circuit.SetActiveBus(AllBuses[k1])
#         tempVoltagePU = dss.Bus.puVmagAngle()
# #         print(tempVoltagePU)
# #         print(tempVoltage)
# #         print(tempVoltage[::2])
#         tempVoltagePU = tempVoltagePU[::2]
#         tempVoltagePU = np.sum(tempVoltagePU)/len(tempVoltagePU)
# #         print(tempVoltage)
#         LoadBusVoltage[kt,k1] = tempVoltagePU
    
    # Iterate through all buses, store voltage
    for k1 in range(len(AllBuses)):
        dss.Circuit.SetActiveBus(AllBuses[k1])
        tempVoltagePU = dss.Bus.puVmagAngle()[::2]
#         print(AllBuses[k1])
#         print(tempVoltage)
        tempVoltagePU = np.sum(tempVoltagePU)/len(tempVoltagePU)
#         tempVoltagePU = tempVoltage/(dss.Bus.kVBase()*1000)        
        AllBusVoltage[kt,k1] = tempVoltagePU
        
#         print(dss.Bus.VMagAngle())
        
#         if kt == simStartTimeStep + 20:
#             print(AllBuses[k1], tempVoltagePU)
        
#     for k1 in range(len(LoadBuses)):
        
#         inverters[k1].measure_voltage(kt,LoadBusVoltage[kt,k1])
#         inverters[k1].lowpass(kt)
    
#     # Iterate through inverters
    for k1 in range(len(inverters)):
        
        AllBuses.index(inverters[k1].nodename)
        
        inverters[k1].measure_voltage(kt,AllBusVoltage[kt,AllBuses.index(inverters[k1].nodename)])
        
        inverters[k1].lowpass(kt)
        
        if kt >= 0:            
            inverters[k1].compute_reactive_power_output(kt, inverters[k1].Vlp[kt])
        
#         # Get index of bus at which inverter is located - fix this
#         AllBuses.index(inverters[k1].nodename[1:4])
        
#         # inverter takes local voltage measurement
#         inverters[k1].measure_voltage(kt,AllBusVoltage[kt,AllBuses.index(inverters[k1].nodename[1:4])])
#         # inverter lowpass filters voltage
#         inverters[k1].lowpass(kt)
        
#         # set reactive power setpoint to 0 for first 10 time steps (to allow lowpass filter to ramp up)
#         if kt >= simStartTimeStep + 10:            
#             inverters[k1].compute_reactive_power_output(kt, inverters[k1].Vlp[kt])
        
# #         inverters[k1].compute_reactive_power_output(kt, inverters[k1].Vlp[kt])

    for k1 in range(len(escs)):
        escs[k1].observer(kt, AllBusVoltage[kt,AllBuses.index(escs[k1].nodename)])
        escs[k1].esc_function(kt)

    # Attack inverters at this timestep
    if kt == 31:
        for k1 in range(len(inverters)):
            # Set new Volt-VAr curve breakpoints
            inverters[k1].set_VBP(inverters[k1].VBP)
            if np.random.rand() <= 2:
                print('~~~~~')
                print(time[kt])
                print(inverters[k1].nodename)
                print(inverters[k1].VBP)
                inverters[k1].hackFlag = True
                inverters[k1].set_VBP(np.array([0.98, 0.99, 1.01, 1.02]))
                inverters[k1].set_VBP(np.array([0.99, 0.999, 1.001, 1.01]))
#                 inverters[k1].set_VBP(np.array([0.998, 0.999, 1.001, 1.002]))
#                 inverters[k1].set_VBP(np.array([0.997, 0.998, 0.999, 1.000]))
#                 inverters[k1].set_VBP(np.array([0.996, 0.997, 0.998, 0.999]))
#                 inverters[k1].set_VBP(np.array([0.995, 0.996, 0.997, 0.998]))
#                 inverters[k1].set_VBP(np.array([0.99, 0.999, 1.001, 1.01])-0.01)
#                 print('~~~~~')
#                 print(time[kt])
#                 print(inverters[k1].nodename)
                print(inverters[k1].VBP)


                

Initial Model Converged. Proceeding to Next Step.
['load_a02_a', 'load_a03_a', 'load_a04_a', 'load_a06_a', 'inv_a01_a', 'inv_a02_a', 'inv_a03_a', 'inv_a04_a', 'inv_a05_a', 'inv_a06_a', 'esc_a01_a', 'esc_a02_a', 'esc_a03_a', 'esc_a04_a', 'esc_a05_a', 'esc_a06_a']
[150. 150. 150. 150.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.]
[50. 50. 50. 50.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[1.05441238 1.03266504 1.0255829  0.96485312 1.04010997 1.05318714
 0.9278077  0.93195476 1.07170619 1.09822971 1.00907842 0.91299326
 1.08876576 0.93946594 1.02208115 0.91513794]

0.00
~~~~~
31.0
a03
[0.95 0.98 1.02 1.05]
[0.99  0.999 1.001 1.01 ]
~~~~~
31.0
a05
[0.95 0.98 1.02 1.05]
[0.99  0.999 1.001 1.01 ]
0.01
0.02


/home/michael/miniconda3/envs/pycigar/lib/python3.7/site-packages/ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
font = {'weight':'bold', 'size':12}

plt.rc('font', **font)

plotStartTimeStep = 0
plotEndTimeStep = len(time)

# plotStartTimeStep = 12*3600 + 1
# plotEndTimeStep = 12*3600 + 1 + 300

# time = np.arange(plotStartTimeStep,plotEndTimeStep)

# timehh = ((time-1)/3600).astype(int)
# timemm = ((time-1) - 3600*timehh)/60

# timeplot = timehh.astype(float) + timemm/100

fig = plt.figure(figsize=[18,6])
# for k1 in range(0,LoadBusVoltage.shape[1]):
for k1 in range(len(LoadBuses)):
#     plt.plot(time,LoadBusVoltage[:,k1])
    plt.plot(time,LoadBusVoltage[:,k1] - np.mean(LoadBusVoltage[:,k1]))
#     print(LoadBuses[k1])
plt.title('Node Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage [p.u.]')
plt.show()


fig = plt.figure(figsize=[18,6])
# for k1 in range(0,LoadBusVoltage.shape[1]):
for k1 in range(len(AllBuses)):
    plt.plot(time,AllBusVoltage[:,k1],label=AllBuses[k1])
#     print(LoadBuses[k1])
plt.title('Node Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage [p.u.]')
plt.legend()
plt.show()

fig = plt.figure(figsize=[18,6])
for k1 in range(0,len(inverters)):
#     plt.plot(inverters[k1].Vmeas[:])
    plt.plot(time,inverters[k1].Vmeas[:],'--')
    plt.plot(time,inverters[k1].Vlp[:])
# plt.ylim([0.85, 1.05])
plt.title('Inverter Low-pass Filter Voltage')
plt.xlabel('Time')
plt.ylabel('Voltage [p.u.]')
plt.show()

fig = plt.figure(figsize=[18,6])
for k1 in range(0,len(inverters)):
    plt.plot(time,inverters[k1].reactive_power[:])
# plt.ylim([-100, 100])
plt.title('Inverter Reactive Power Output')
plt.xlabel('Time')
plt.ylabel('Percent Capacity [%]')
plt.show()

# fig = plt.figure(figsize=[18,6])
# plt.plot(time,PloadTS[:,:])
# plt.plot(time,QloadTS[:,:])
# plt.title('Load Demands')
# plt.xlabel('Time')
# plt.ylabel('Percent Capacity [%]')
# plt.show()

fig = plt.figure(figsize=[18,6])
plt.subplot(2,1,1)
plt.plot(time,PloadTS[:,:])
plt.title('Node Active Power Demand')
# plt.xlabel('Time')
plt.ylabel('Active Power Demand [kW]')
plt.subplot(2,1,2)
plt.plot(time,QloadTS[:,:])
plt.title('Node Reactive Power Demand')
plt.xlabel('Time')
plt.ylabel('Reactive Power Demand [kVAr]')
plt.show()

# fig = plt.figure(figsize=[18,6])
# plt.plot(time,PinjTS[:,:])
# plt.plot(time,QinjTS[:,:])
# plt.title('Injection')
# plt.show()

fig = plt.figure(figsize=[18,6])
plt.subplot(2,1,1)
plt.plot(time,PinvTS[:,:])
plt.title('Node Active Power Injection')
# plt.xlabel('Time')
plt.ylabel('Active Power Injection [kW]')
plt.subplot(2,1,2)
plt.plot(time,QinvTS[:,:])
plt.title('Node Reactive Power Injection')
plt.xlabel('Time')
plt.ylabel('Reactive Power Injection [kVAr]')
plt.show()

# fig = plt.figure(figsize=[18,6])
# plt.plot(time,PtotalTS[:,:])
# plt.plot(time,QtotalTS[:,:])
# plt.title('Total')
# plt.show()

fig = plt.figure(figsize=[18,6])
plt.subplot(2,1,1)
plt.plot(time,PtotalTS[:,:])
plt.title('Node Total Active Power')
# plt.xlabel('Time')
plt.ylabel('Total Active Power [kW]')
plt.subplot(2,1,2)
plt.plot(time,QtotalTS[:,:])
plt.title('Node Total Reactive Power')
plt.xlabel('Time')
plt.ylabel('Total Reactive Power [kVAr]')
plt.show()

fig = plt.figure(figsize=[18,9])
plt.subplot(4,1,1)
plt.plot(time,escs[0].y1)
plt.title('y1')
# plt.xlabel('Time')
plt.ylabel('y1')
plt.subplot(4,1,2)
plt.plot(time,escs[0].y2)
plt.title('y2')
plt.ylabel('y2')
plt.subplot(4,1,3)
plt.plot(time,escs[0].y3)
plt.title('y3')
plt.ylabel('y3')
plt.subplot(4,1,4)
plt.plot(time,escs[0].y4)
plt.title('y4')
plt.xlabel('Time')
plt.ylabel('y4')
plt.show()

fig = plt.figure(figsize=[18,15])
plt.subplot(6,1,1)
plt.plot(time,escs[0].y4)
plt.title('objective')
plt.ylabel('y4')
plt.subplot(6,1,2)
plt.plot(time,escs[0].rho)
plt.title('rho')
plt.ylabel('rho')
plt.subplot(6,1,3)
plt.plot(time,escs[0].sigma)
plt.title('sigma')
plt.ylabel('sigma')
plt.subplot(6,1,4)
plt.plot(time,escs[0].xi)
plt.title('xi')
plt.ylabel('xi')
plt.subplot(6,1,5)
plt.plot(time,escs[0].uhat)
plt.title('uhat')
plt.ylabel('uhat')
plt.subplot(6,1,6)
plt.plot(time,escs[0].u)
plt.title('u')
plt.xlabel('Time')
plt.ylabel('u')
plt.show()

# print(timeplot)
